In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../TOEFL_11/index.csv')
df.head()

,Unnamed: 0,Filename,Prompt,Language,Score Level,total word,distance,Hpoint,xmin,alpha,...,sigma,logtrue,file,pl_s,pl_C,pl_R2,log_d,log_m,log_k,log_R2
0,0,88.txt,P6,KOR,high,385,6.000000,8,2.0,2.273182,...,3.025490,0,88.txt,0.626725,27.346485,0.883300,3.478245,0.639538,1.090635,0.869275
1,1,278.txt,P6,DEU,medium,321,11.000000,7,3.0,2.421178,...,1.817156,0,278.txt,0.720874,31.641664,0.962596,3.573915,0.757765,1.032666,0.951214
2,2,348.txt,P1,TUR,high,363,8.306624,6,2.0,2.398038,...,1.739209,0,348.txt,0.621470,21.737883,0.965709,3.352892,0.874682,0.849857,0.889519
3,3,666.txt,P2,ZHO,medium,362,4.690416,6,5.0,3.410995,...,0.746383,0,666.txt,0.538645,18.320499,0.924139,2.824981,0.296708,1.441791,0.969115
4,4,733.txt,P6,TEL,medium,344,10.630146,7,6.0,3.124099,...,1.163868,0,733.txt,0.646729,25.998149,0.938335,3.393319,0.638436,1.093227,0.926643


In [9]:
df =  df[['file', 'Score Level']]
df.head()

,file,Score Level
0,88.txt,high
1,278.txt,medium
2,348.txt,high
3,666.txt,medium
4,733.txt,medium


In [14]:
contents = []

for file in df['file'].values:
    f=open("../TOEFL_11/essays/88.txt", "r")
    content =f.read()
    contents.append(content)

"Some people might think that traveling in a group led by a tour guide is a good way. But, a group tour normally has its strict schedule and does not offer the tourists in group any chance to meet many people in new place. \nSo, I strongly disagree with the statement that the best way to travel is in a group led by a tour guide. \n\nFirst, a group tour has its own strict schedule and all the tour team members should follow it. And it means that I can not choose the place where I want go and the food which I want to eat. For example, when I went to Thailand for the first time, I chose a group tour because I was young and it was the first time to go abroad. I really wanted to go many attractive shops and spots in the street in Bangkok, but I couldn't go there because I was in a group tour and I had to follow the schedule which the tour company offered. And I had to wake up early in the morning and had to go back to hotel in early evening, even though I wanted to enjoy more sleep in the b

In [15]:
df['eassy'] = contents

In [16]:
df.head()

,file,Score Level,eassy
0,88.txt,high,Some people might think that traveling in a gr...
1,278.txt,medium,Some people might think that traveling in a gr...
2,348.txt,high,Some people might think that traveling in a gr...
3,666.txt,medium,Some people might think that traveling in a gr...
4,733.txt,medium,Some people might think that traveling in a gr...


In [17]:
import numpy as np
import nltk
import re
from nltk.corpus import stopwords
from gensim.models import Word2Vec

def essay_to_wordlist(essay_v, remove_stopwords):
    """Remove the tagged labels and word tokenize the sentence."""
    essay_v = re.sub("[^a-zA-Z]", " ", essay_v)
    words = essay_v.lower().split()
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    return (words)

def essay_to_sentences(essay_v, remove_stopwords):
    """Sentence tokenize the essay and call essay_to_wordlist() for word tokenization."""
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    raw_sentences = tokenizer.tokenize(essay_v.strip())
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append(essay_to_wordlist(raw_sentence, remove_stopwords))
    return sentences

def makeFeatureVec(words, model, num_features):
    """Make Feature Vector from the words list of an Essay."""
    featureVec = np.zeros((num_features,),dtype="float32")
    num_words = 0.
    index2word_set = set(model.wv.index2word)
    for word in words:
        if word in index2word_set:
            num_words += 1
            featureVec = np.add(featureVec,model[word])        
    featureVec = np.divide(featureVec,num_words)
    return featureVec

def getAvgFeatureVecs(essays, model, num_features):
    """Main function to generate the word vectors for word2vec model."""
    counter = 0
    essayFeatureVecs = np.zeros((len(essays),num_features),dtype="float32")
    for essay in essays:
        essayFeatureVecs[counter] = makeFeatureVec(essay, model, num_features)
        counter = counter + 1
    return essayFeatureVecs

ModuleNotFoundError: No module named 'gensim'